<a href="https://colab.research.google.com/github/andyslater/Python-Fundamentals/blob/master/IWM_modeler_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Init

In [ ]:
# GPU install video https://www.youtube.com/watch?v=PitcORQSjNM
"""
first get a runtime with a GPU
  - Runtime / factory reset runtime
  - Runtime / change runtime type
"""
!pip install tensorflow-gpu

import tensorflow as tf
from tensorflow import keras
print("tensorflow version: ",tf.__version__)
print("keras version: ",keras.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install import-ipynb
import import_ipynb



In [ ]:
%cd /content/drive/MyDrive/'Colab Notebooks'
from utils import *
%cd /content


In [ ]:
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import text
import numbers



pd.options.display.float_format = '{:,.4f}'.format
pd.set_option("display.precision", 2)

## read in the data

In [ ]:
location = '/content/drive/MyDrive/UCONN/ML Independent Study/project2/data/'
options_file = "IWM Options All.csv"
stocks_file = "IWM Stocks All.csv"


stocks0 = pd.read_csv(f"{location}{stocks_file}")
options0 = (pd.read_csv(f"{location}{options_file}")).query("open_interest>100 | volume>100")
options0['strike_price'] = options0['strike_price'] /1000.
options0.rename(columns={'best_offer':'ask', 'best_bid':'bid'}, inplace=True) # buyers get ask price, sellers get the bid price

stocks0['date_'] = pd.to_datetime(stocks0['date'].astype(str), format='%Y%m%d')
stocks = stocks0


In [ ]:
# display(stocks0)
# options0
# display(options0.columns)
# display(stocks0.columns)


In [ ]:

# combine option and stock price, keep only needed cols
options_field_list = 'date,ticker,cp_flag,exdate,strike_price,bid,ask,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid'.split(',')
for k in 'low,high,open,close,volume'.split(','): stocks[f"stock_{k}"] = stocks[k] 
stock_field_list ='date,stock_low,stock_high,stock_open,stock_close,stock_volume'.split(',')
options = pd.merge(left=options0[options_field_list], right=stocks[stock_field_list], how='left', on='date', sort=False).sort_values(by='date,ticker,cp_flag,exdate,strike_price'.split(',')).reset_index(drop=True)

# remove cols where any key field is Nan
st = len(options)
options.dropna(subset='date,ticker,cp_flag,strike_price,bid,ask,volume,open_interest,stock_open'.split(','), inplace=True)
en = len(options)
if st != en: print(f"{st-en} rows removed becuase key data is missing from one or more fields")
print(f"{len(options)} rows in options")

# calc the number of days till exp
options['date_'] = pd.to_datetime(options['date'].astype(str), format='%Y%m%d')
options['exdate_'] = pd.to_datetime(options['exdate'].astype(str), format='%Y%m%d')
options['days'] = (options['exdate_'] - options['date_']).dt.days #https://stackoverflow.com/questions/22132525/add-column-with-number-of-days-between-dates-in-dataframe-pandas/45039811
#options.drop(['date_','exdate_'],axis=1,inplace=True)


In [ ]:
options

In [ ]:


def show_chart(table=pd.DataFrame(),exp_offset=50,shade=None):
  # show the stock trend with markers for the options exdates
  if len(table) == 0: table = stocks
  ax = plot_line(table,'date_', 'close', title=f"IWM close price and options exdates", xlabel='date / exdate',ylabel='price',figsize=[40,3],show=0)

  # overlay the exdates data
  min,max = table.date.min(),table.date.max()
  exdates_ = options.query(f"exdate>={min} & exdate<={max}").exdate_.unique()
  min = table.close.min()
  for k in exdates_:  
    d = f"{k}"[:10]
    plt.axvline(x=k, color='g', linestyle='--', linewidth=.75)
    plt.text(k, min-exp_offset, d, rotation=90, verticalalignment='center')

  if shade: 
    print(type(table['date_'][0]), pd._libs.tslibs.timestamps.Timestamp)
    for i in [0,1]: shade[i] = pd.to_datetime(str(shade[i]), format='%Y%m%d') if type(shade[i]) != pd._libs.tslibs.timestamps.Timestamp else shade[i]
    plt.axvspan(shade[0], shade[1], facecolor='b', alpha=0.07)

  plt.show()



# Options Object

In [ ]:
  class Options():
    def __init__(self,options,date=20200513, ticker='IWM', exdate=20200605, change_range=None, id=None, end_date=None, verbose=0):
      if id==None: id = date
      self.Timer = Timer()
      self.verbose = verbose
      self.id = id
      self.date = date
      self.ticker = ticker
      self.exdate = exdate
      self.change_range = change_range
      self.commission = 1.2
      self.options = options
      self.positions = pd.DataFrame(columns='date,ticker,exdate,buy_strike,buy_ask,sell_strike,sell_bid,name'.split(','))
      self.all_positions = pd.DataFrame()
      self.cum_outcome = pd.DataFrame()
      self.bin_size = 10

    def set_state(self,date=None,ticker=None,exdate=None):
      if date: self.date = date
      if ticker: self.ticker = ticker
      if exdate: self.exdate = exdate
    def reset_positions(self): self.positions = pd.DataFrame(columns='date,ticker,exdate,buy_strike,buy_ask,sell_strike,sell_bid,name'.split(','))
    def reset_all_positions(self): self.all_positions = pd.DataFrame()
    def id2name(self,optionid,fields=None):
      if fields==None: fields = 'date,ticker,exdate,cp_flag,strike_price'
      d = self.options.query(f"optionid=={optionid}").to_dict('records')[0]
      ret = ""
      for f in fields.split(','): ret += f"{d[f]:0.2f} " if type(d[f])==float else f"{d[f]} "
      return ret[:-1]
    def pname(self): return f"{str(self.date)[4:]} {self.ticker} {str(self.exdate)[4:]}" 
    def position2name(self,d,fields=None):
      if fields==None: fields = 'date,ticker,exdate,buy_strike,sell_strike'
      ret = ""
      for f in fields.split(','): ret += f"{d[f]:0.2f} " if type(d[f])==float else f"{d[f]} "
      return ret[:-1]

    def dfsave(self,df,name,folder=None):
      path = f"/content/drive/MyDrive/{folder}/"
      df.to_csv(name, index=False)
      !cp $name $path # assumes drive has been mounted
    def dfrestore(self,name,folder=None): return pd.read_csv(f"/content/drive/MyDrive/{folder}/{name}")
    def save_state(self,id=None):
      if id==None: id = self.id
      f = f"Options_{id}_"
      folder = 'COLAB_SAVE'
      self.dfsave(self.positions,f"{f}positions.csv", folder=folder)
      self.dfsave(self.all_positions,f"{f}all_positions.csv", folder=folder)
      print(f"saved state to {folder}/{f}*")
    def restore_state(self,id=None):
      if id==None: id = self.id
      print(f"restoring state {id}")
      self.positions = self.dfrestore(f"Options_{id}_positions.csv",folder='COLAB_SAVE')
      self.all_positions = self.dfrestore(f"Options_{id}_all_positions.csv",folder='COLAB_SAVE')



    def get_PUT_strikes(self,date=None,ticker=None,exdate=None,strikes=None): return self.get_strikes(date,ticker,exdate,strikes=strikes,cp_flag='P')
    def get_CALL_strikes(self,date=None,ticker=None,exdate=None,strikes=None): return self.get_strikes(date,ticker,exdate,strikes=strikes,cp_flag='C')
    def get_strikes(self,date,ticker,exdate,cp_flag=None,strikes=None):
      self.set_state(date,ticker,exdate)
      ret = self.options.query(f"date=={self.date} & ticker=='{self.ticker}' & exdate=={self.exdate}")
      if cp_flag: ret = ret.query(f"cp_flag=='{cp_flag}'")
      if strikes != None: 
        if type(strikes) == str: strikes = strikes.split(',')
        elif isinstance(strikes, numbers.Number): strikes =[strikes,]
        ret = ret[ret.strike_price.isin(strikes)]
      return ret

    def get_exdates(self,date=None,max_exdate=None,ticker=None):
      self.set_state(date,ticker)
      ret = options.query(f"date=={self.date} & ticker=='{self.ticker}'")
      if max_exdate: ret = ret.query(f"exdate <= {max_exdate}")
      return ret.exdate.unique()

    def get_dates(self,start_date=None,end_date=None,cnt=None,step=1):
      p = self.options
      if start_date: p = p.query(f"date>={start_date}")
      if end_date: p = p.query(f"date<={end_date}")
      p = p['date'].unique()
      p.sort()
      if cnt: p = p[:cnt*step:step]
      return p

    def add_bps_position(self,date=None,ticker=None,exdate=None,buy_strike=None,sell_strike=None):
      self.set_state(date,ticker,exdate)
      if not buy_strike or not sell_strike: 
        print("buy and sell strike required")
        return
      idx = len(self.positions)
      # add only if its not already there
      present = len(self.positions.query(f"date=={self.date} & ticker=='{self.ticker}' & exdate=={self.exdate} &buy_strike=={buy_strike} & sell_strike=={sell_strike}")) >0
      if not present: 
        name = f"{str(self.date)[4:]} {self.ticker} {str(self.exdate)[4:]} {buy_strike:0.2f} {sell_strike:0.2f}"
        self.positions.loc[idx] = {'name': name, 'date': self.date, 'ticker': self.ticker, 'exdate': self.exdate, 'buy_strike': buy_strike, 'sell_strike': sell_strike }

    # loop over all possible combinations for the given date,ticker and exdate
    def OLD_make_position_table(self,limit=0,reset_all=1,add2all=0, verbose=0):
      self.Timer.start('make_position_table')
      if reset_all:
        self.reset_all_positions()
        self.reset_positions()
      #put_strikes = self.get_PUT_strikes().query("open_interest>100 | volume>100")
      put_strikes = self.get_PUT_strikes()
      sell_strikes =  put_strikes.strike_price.unique()[:-1]
      if verbose: print(f"{len(sell_strikes)+1} strikes for {self.date} {self.ticker} {self.exdate}")
      i = 0
      for ss in sell_strikes:
        buy_strikes =  put_strikes.query(f"strike_price>{ss}").strike_price.unique()
        for bs in buy_strikes: 
          #if i%100==0: print(ss,bs, end='\r')
          self.add_bps_position(buy_strike=bs,sell_strike=ss)
          i += 1
          if limit and i>=limit: break
        if limit and i>=limit: break
      if verbose: print(f"{i} positions added")
      self.Timer.stop('make_position_table')
      return

    
    # loop over all possible combinations for the given date,ticker and exdate
    def make_position_table(self,limit=0,reset_all=1,add2all=0, verbose=0):
      self.Timer.start('make_position_table')
      if reset_all:
        self.reset_all_positions()
        self.reset_positions()
      #put_strikes = self.get_PUT_strikes().query("open_interest>100 | volume>100")
      put_strikes = self.get_PUT_strikes()
      sell_strikes =  put_strikes.strike_price.unique()[:-1]
      if verbose: print(f"{len(sell_strikes)+1} strikes for {self.date} {self.ticker} {self.exdate}")
      i = 0
      dlist = []
      for ss in sell_strikes:
        buy_strikes =  put_strikes.query(f"strike_price>{ss}").strike_price.unique()
        for bs in buy_strikes: 
          #if i%100==0: print(ss,bs, end='\r')
  #        self.add_bps_position(buy_strike=bs,sell_strike=ss)
          name = f"{str(self.date)[4:]} {self.ticker} {str(self.exdate)[4:]} {bs:0.2f} {ss:0.2f}"
          dlist.append( {'name': name, 'date': self.date, 'ticker': self.ticker, 'exdate': self.exdate, 'buy_strike': bs, 'sell_strike': ss } )
          i += 1
          if limit and i>=limit: break
        if limit and i>=limit: break
      if verbose: print(f"{i} positions added")
      dfadd = pd.DataFrame(dlist)
      self.positions = pd.concat([self.positions,dfadd],axis=0).reset_index(drop=True).drop_duplicates()    
      self.Timer.stop('make_position_table')
      return



    def update_bps_position(self,plot_range=None,add2all=0, verbose=0):
      self.Timer.start('update_bps_position')
      for loopbreak in [1]:
        if len(self.positions) == 0:
          print("no positions")
          break

        pfields = 'date,ticker,exdate,buy_strike,sell_strike,name'.split(',')
        ofields ='date,ticker,stock_open,exdate,strike_price,optionid'.split(',')

        # get the option SELL data
        strikes = list(self.positions.sell_strike.unique()) # get the sell strikes for all exp
        puts = self.get_PUT_strikes(strikes=strikes) # get all the puts to so we can add this data to the positions
        self.positions = pd.merge(left=self.positions[pfields], right=puts[ofields+['bid','ask']], how='left', left_on='date,ticker,exdate,sell_strike'.split(','), right_on='date,ticker,exdate,strike_price'.split(',') ,sort=False)
        self.positions.rename(columns={'optionid':'sell_optionid','bid':'sell_bid','ask':'sell_ask'}, inplace=True)

        # could be the case where a SELL strike_price was added to the position list, but there is no option. So remove them
        # x = len(self.positions.query("sell_bid != sell_bid")) # rows that aare Nan
        # print('SELL NAN=',x)
        self.positions.dropna(inplace=True)
        if len(self.positions) == 0: break

        # calc the change% 
        self.positions['change_pc'] = 100*(self.positions.sell_strike/self.positions.stock_open -1).astype(float) # must do astype becasue of bug, otherwise .query("change_pc>-1.5") generates an error https://stackoverflow.com/questions/50400843/using-negative-numbers-in-pandas-dataframe-query-expression

        # remove any rows that do not meet the change_range - dont want to computer and store garbage
        st = len(self.positions)
        if self.change_range: self.positions = self.positions.query(f"(change_pc <= {self.change_range[1]}) & (change_pc >= {self.change_range[0]})")
        fn = len(self.positions)
        if verbose and st!=fn: print(f"{fn} rows in sell_strikes after {st-fn} rows removed because change_pc is not within the change_range {self.change_range}")
        if not fn: break

        # fill optoin BUY strike data
        strikes = list(self.positions.buy_strike.unique()) # get the sell strikes for all exp
        puts = self.get_PUT_strikes(strikes=strikes) # get all the puts to so we can add this data to the positions
        self.positions = pd.merge(left=self.positions[pfields +'sell_bid,sell_ask,change_pc,sell_optionid'.split(',')], right=puts[ofields+['bid','ask','days',]], how='left', left_on='date,ticker,exdate,buy_strike'.split(','), right_on='date,ticker,exdate,strike_price'.split(',') ,sort=False)
        self.positions.rename(columns={'optionid':'buy_optionid','bid':'buy_bid','ask':'buy_ask'}, inplace=True)

        # could be the case where a BUY strike_price was added to the position list, but there is no option. So remove them
        #x = len(self.positions.query("buy_bid != buy_bid")) # rows that aare Nan
        #print(f"removing {x} rows where buy strike_price not found in options list")
        self.positions.dropna(inplace=True)
        if len(self.positions) == 0: break

        # rename 
        #self.positions.dropna(inplace=True, axis=0) # should never need to do this
        self.positions.rename(columns={'stock_open':'stock_price'}, inplace=True)

        # calc remaining stuff
        self.positions['cost'] = 100*(self.positions.buy_ask - self.positions.sell_bid) + self.commission
        self.positions['max_profit'] = 100*(self.positions.buy_strike -self.positions.sell_strike) -self.positions.cost
        self.positions['max_ROI_pc'] = (100 * self.positions.max_profit / self.positions.cost).astype(int)

        # REMOVE CRAZY STUFF
        self.positions = self.positions.query("cost<=350")

        # reorder and sort fields
        self.positions = self.positions['date,ticker,stock_price,exdate,days,buy_strike,sell_strike,buy_ask,sell_bid,buy_bid,sell_ask,cost,max_profit,max_ROI_pc,change_pc,buy_optionid,sell_optionid,name'.split(',')].sort_values(by='change_pc,max_ROI_pc'.split(',')).reset_index(drop=True)
        
        if plot_range: self.plot_table(change_range=plot_range)
        if add2all: 
          if len(self.all_positions) == 0: self.all_positions = self.positions.copy()
          else: 
            self.all_positions = self.all_positions.query( f"not ( date=={self.date} & ticker=='{self.ticker}' & exdate=={self.exdate})" )  # delete all currently inall
            self.all_positions = self.all_positions.append(self.positions,ignore_index=True)
      self.Timer.stop('update_bps_position')
      return 

    def get_options_from_positions(self,table=pd.DataFrame()):
      # stack the buy and sell stikes so they are not combined in any rows. just get all options involved in any side of the bps
      if len(table) == 0: table = self.all_positions
      df1 = table.groupby('date,ticker,exdate,buy_strike'.split(',')).size().reset_index(name='F1').rename(columns={'buy_strike':'strike_price'})
      df2 = table.groupby('date,ticker,exdate,sell_strike'.split(',')).size().reset_index(name='F2').rename(columns={'sell_strike':'strike_price'})
      df = pd.merge(df1,df2, indicator=True, how='outer', on='date,ticker,exdate,strike_price'.split(',')).query('_merge=="left_only"').drop('_merge,F1,F2'.split(','), axis=1)
      # merge back the options data
      df = pd.merge(df,self.options[self.options.cp_flag=='P'], how='left', on='date,ticker,exdate,strike_price'.split(','))
      return df

    def make_best_positions(self,dates,max_exdate_distance=30,limit=0,verbose=0):
      self.Timer.start('make_best_positions')
      # loop though all the dates building all possible combinations of BPS within the expiration date limits
      # select only the best postions for each date
      p = pd.DataFrame()
      i = 0
      for date in dates: # this is the date in time when we are placing the bet
        i += 1
        self.set_state(date=date) # set the new date
        max_exdate = add_days(date, max_exdate_distance)
        exdates = self.get_exdates(max_exdate=max_exdate)
        if verbose: hprint(f"\nfor {date} there are {len(exdates)} exdates within {max_exdate_distance} days: {exdates}") 
        if verbose>1: print(f"date: {date}, max_exdate: {max_exdate}, exdates={exdates}")

        for exdate in exdates:
          if exdate == date: continue
          if verbose: print(f"\nexp={exdate}")
          self.reset_positions() # clear out positions
          self.set_state(date=date,exdate=exdate) # set the new exdate
          self.make_position_table(limit=limit,reset_all=0, verbose=verbose) # all combinations for the date, ticker and exdate
          self.update_bps_position(add2all=1, verbose=verbose) # add in the options data
        #  op.plot_positions_ROI(table=op.all_positions)

        p = self.all_positions

        if False:
          # select only the best from each exdate for this date
          if len(self.all_positions) == 0:
            hprint(f"there are no positions that meet the requirements dates={dates}, max_exdate_distance={max_exdate_distance}, limit={limit}")
            break
          fields = 'date,exdate,change_pc'.split(',')
          st = len(p)
          p = p.loc[p.groupby(fields)['max_ROI_pc'].idxmax()] # keep only the max value fo each groupby
          en = len(p)
          if verbose>1 and st != en: print(f"{st-en} total positions removed because they are not optimal")

        print(f"finished date={date} ({i}/{len(dates)}) total positions={len(p)} elapsed_time={self.Timer.etime()}")
        if verbose: self.Timer.report()
      self.Timer.stop('make_best_positions')
      return p

    def make_derived(self,t=pd.DataFrame()):
      # derived data is also calculated elsewhere earlier in the processes. This is the final 
      if len(t) == 0: t = self.positions
      t['strike_spread'] = t['buy_strike'] - t['sell_strike']
      t['stock_spread'] = t['buy_strike'] - t['stock_price']
      t['buy_bid_ask_spread'] = 100 * (t['buy_ask'] - t['buy_bid']) / t['buy_strike']
      t['sell_bid_ask_spread'] = 100 * (t['sell_ask'] - t['sell_bid']) / t['sell_strike']

    def update_position_outcome(self,positions,verbose=0):
      # given the positions, calculate the outcome
      commission = self.commission
      # merge in the stock close price
      ptable = pd.merge(positions,stocks0['date,stock_close'.split(',')].rename(columns={'date':'exdate'}), on='exdate', sort=False)
      # three scenarios for excercise , stock is below sell_strike, above_buy_strike, or betweek
      ptable.loc[ptable.stock_close<=ptable.sell_strike, 'outcome'] = 100*(ptable['buy_strike'] - ptable['sell_strike'] - (ptable['buy_ask'] -ptable['sell_bid']) ) -commission
      ptable.loc[ptable.stock_close>=ptable.buy_strike, 'outcome'] = -100*(ptable['buy_ask'] -ptable['sell_bid'])  -commission
      ptable.loc[(ptable.stock_close<ptable.buy_strike) & (ptable.stock_close>ptable.sell_strike), 'outcome'] = 100*(ptable['buy_strike'] - ptable['stock_close'] - (ptable['buy_ask'] -ptable['sell_bid']) )-commission
      # sort it
      ptable = ptable['date,ticker,exdate,outcome,days,stock_price,stock_close,buy_strike,sell_strike,buy_ask,sell_bid,buy_bid,sell_ask,cost,max_profit,max_ROI_pc,change_pc,buy_optionid,sell_optionid,name'.split(',')].sort_values(by='outcome,date,name'.split(','))
      #hprint('ptable'); display(ptable)
      if verbose: 
        min, max = positions.date.min(), positions.exdate.max()
        hprint(f"outcome: {ptable.outcome.sum():0.2f}, min exdate: {min}, max exdate: {max}, positions:{len(positions)}")
      return ptable

    def make_cum_outcome(self,results=pd.DataFrame(),bin_size=None):
      if bin_size: self.bin_size = bin_size
      if len(results)==0: results = self.positions
      # calc cum distribution
      outcome = []
      roi = []
      max_outcome = results.outcome.sum()
      for r in range(int(2000/self.bin_size)+1):
        cum_roi = r*self.bin_size
        roi.append(cum_roi)
        cum_outcome = results[results.max_ROI_pc<cum_roi].outcome.sum()
        outcome.append(cum_outcome)
        if cum_outcome/max_outcome > 0.95: 
          #print("breaking", r, cum_outcome, max_outcome)
          break # dont need to see more than this
      cd = pd.DataFrame()
      cd['roi'] = roi
      cd['outcome'] = outcome
      self.cum_outcome = cd
      max_cum_roi = cd['outcome'].idxmax()
      hprint(f"\noutcome is increasing until {max_cum_roi*self.bin_size} with a cumulative outcome of ${cd.outcome[max_cum_roi]:0.0f}")
      self.plot_cum_outcome()


    def build_dataset(self,start_date=None, end_date=None, max_days=None, step_days=1, limit=0,positions=pd.DataFrame()):
      self.Timer.start('build_dataset')
      # given inputs, make all the postions, calculate the outcomes, plot to retuls, along with the cumulative outcome v. max_ROI_pc
      dates = self.get_dates(start_date=start_date, end_date=end_date, cnt=max_days, step=step_days) 
      hprint(f"Processing {len(dates)} dates: {dates}")
      # make all the positions, select the best ones
      if len(positions) == 0: self.positions = self.make_best_positions(dates,max_exdate_distance=max_exdate_distance,limit=limit, verbose=0)
      else: self.positions = positions
      hprint(f"{len(self.positions)} postions of {len(self.all_positions)} selected")
      # update the outcome and addtional derived data for the positions
      self.positions = self.update_position_outcome(self.positions)
      make_derived(self.positions) #
      self.save_state()
      
      # output the plots
      min, max = self.positions.date.min(), self.results.exdate.max()
      hprint(f"outcome: {self.positions.outcome.sum():0.0f}, min exdate: {min}, max exdate: {max}, positions:{len(self.positions)}")
      plot_points(self.positions,'max_ROI_pc',fields='outcome',title='outcome v. max_ROI_pc', ylabel='Outcome ($)',xlabel='max_ROI_pc',figsize=[20,5])
      self.make_cum_outcome(self.positions)
      self.Timer.stop('build_dataset')
      self.Timer.report()

    def plot_cum_outcome(self,cum_outcome=pd.DataFrame()): 
      if len(cum_outcome)==0 : cum_outcome = self.cum_outcome
      if len(cum_outcome)==0: cum_outcome = self.cum_outcome = self.make_cum_outcome()
      plot_points(cum_outcome,'roi',fields='outcome',title='cumulative outcome v. max_ROI_pc', ylabel='cumulative outcome ($)',xlabel='max_ROI_pc',figsize=[20,5])

    def plot_ROI_v_outcome(self,results=pd.DataFrame()):
      if len(results) ==0: results = self.positions
      plot_points(results,'max_ROI_pc',fields='outcome',title='outcome v. max_ROI_pc', ylabel='Outcome ($)',xlabel='max_ROI_pc',figsize=[20,5])

    # was plot_table
    def plot_positions_ROI(self,date=None,ticker=None,exdate=None,change_range=None,xlim=None,ylim=None,title=None,figsize=[20,3],table=pd.DataFrame()):
      # plot the table. x-axis is %change, y-axis ROI%, will plot each exdate. usual should only have one ticker and one date and one exdate in the table
      if len(table)==0: table = self.positions
      fig, ax = plt.subplots(figsize=(figsize))
      #fig = plt.figure()
      if change_range: table = table.query(f"(change_pc <= {change_range[1]}) & (change_pc >= {change_range[0]})")
      if len(table) == 0: return None
      plt.xlabel(f"%change in stock price")
      plt.ylabel('Max ROI %')
      for date in table.date.unique():
        t = table[table.date==date][['change_pc','max_ROI_pc','exdate']]
        exdates = t.exdate.unique()
        for exd in exdates:
          t2 = t[t.exdate==exd]
          plt.plot(t2['change_pc'],t2['max_ROI_pc'],'o', label=f"{date} {self.ticker} {exd}"); 
      plt.grid()
      if title==None: title = f"{self.date} {self.ticker} {self.exdate if len(exdates)==1 else ''}"
      plt.legend()
      plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
      plt.title(title) 
      ax.set_xlim(xlim)
      ax.set_ylim(ylim)
      p = plt.show()
      return table

    def plot_greek(self,table,letter):
      # gien all_positions get the options for all buy_strike and sell_strikes and then plot the greeks over the strike price
      fig, ax = plt.subplots()
      for exp in table.exdate.unique():
        ax.plot( table[table.exdate==exp].strike_price, table[table.exdate==exp][letter], label=f"exdate={exp}")
      ax.set(xlabel='strike_price', ylabel=letter, title=letter)
      ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
      ax.grid()
      plt.show()
    def plot_greeks(self,table=pd.DataFrame()):
      table = self.get_options_from_positions(table)
      for g in 'impl_volatility,delta,gamma,vega,theta'.split(','):
        self.plot_greek(table,g)

    def test(self):
      op = Options(options, date=20200513, ticker='IWM', exdate=20200605, change_range=[-3.1, 0])
      op.add_bps_position(buy_strike=128,sell_strike=123)
      op.add_bps_position(buy_strike=127,sell_strike=123)
      op.update_bps_position()
      print(len(op.positions), len(op.all_positions))
      op.positions
      op.all_positions
      op.plot_greeks()




In [ ]:
show_chart(stocks0)

#generate data set

In [ ]:
if False:
  # start_date = 20190101
  # end_date = 20201231
  step_days = 1
  max_days = None
  max_exdate_distance = 30 # place bets on exdates at most this many days out from date bet is being placed
  op = Options(options, id='generator', date=None, ticker='IWM') #, change_range=[-10,0]
  op.build_dataset(start_date=None, end_date=None, max_days=max_days, step_days=step_days, limit=0)

# Build the Classifier Model

In [ ]:
op = Options(options, id='generator0', date=None)
op.restore_state()
#op.positions.drop(['name'],axis=1)

In [ ]:
op.positions.date.describe()

In [ ]:
from sklearn.model_selection import train_test_split

class OptionModel_1():
  def __init__(self,positions,train_start_date=None, train_days=None, test_start_date=None, test_days=None, verbose=0):
    self.positions = positions

    self.investment = 10000 # total initial investment
    self.max_exp_days = 30 # maximum days till the expiration date (closing the investment)
    self.train_start_date = train_start_date # start training date
    self.train_days = train_days # number of days to train
    self.test_start_date = test_start_date # start of test date (test continues for time_period days)
    self.test_days = test_days # number of days over which test should be run
    self.max_change = None

    # training set
    if self.train_start_date == None: self.train_start_date = self.positions.loc[0]['date']
    if self.train_days == None: self.train_days = 120
    self.train_end_date = sorted(self.positions.date.unique())[self.train_days]
    self.train_set = self.positions.query(f"date<={self.train_end_date}").query(f"exdate<={self.train_end_date}")
    self.train_set = self.date2sequence(self.train_set)
    self.X_train, self.y_train = self.makeXy(self.train_set)

    # test set
    if self.test_start_date == None: self.test_start_date = add_days( self.train_end_date, 1 )
    if self.test_days == None: self.test_days = int(self.train_days/4.0)
    self.test_end_date = sorted(self.positions.date.unique())[self.train_days +self.test_days]
    self.test_set = self.positions.query(f"date>{self.train_end_date} & date<={self.test_end_date}").query(f"exdate<={self.test_end_date}")
    self.test_set = self.date2sequence(self.test_set)
    self.X_test, self.y_test = self.makeXy(self.test_set)

    if verbose:
      print( "state: ", d2print( self.__dict__) )
      print( f"All data: {len(self.positions)} train_set: {len(self.train_set)} test_set: {len(self.test_set)} ") 

    # X_train = d.query("position < 11")[input_fields]
    # X_test = d.query("position >= 11")[input_fields]
    # y_train = d.query("position < 11")[f"anchor{anchor}"]
    # y_test = d.query("position >= 11")[f"anchor{anchor}"]

  def makeXy(self,df):
    normalize_fields = 'days,stock_price,stock_close,buy_strike,sell_strike,buy_ask,sell_bid,sell_ask,cost,max_profit,max_ROI_pc,change_pc,strike_spread,stock_spread,buy_bid_ask_spread,sell_bid_ask_spread'.split(',')
    X = df[normalize_fields].copy()
    y = df['outcome'].copy()
    # how to easily buketize this into say 5 bins so we have ranges of outcomes and then have a softmax as the output 
    y[ y > 0] = .99 # for anything greater then 0 set it to .99, why does this have to be .99
    y[ y < 0] = 0
    dfnormalize_fields(X,normalize_fields)
    return X,y

  def date2sequence(self,t,new_field='day'):
    # add a new field that is simply an incredmenting number for the day
    dates = np.sort(t.date.unique())
    df = pd.DataFrame(columns=['date'],data=dates)
    df[new_field] = df.index
    return t.merge(df,how='left',on='date')



om = OptionModel_1(positions=op.positions.drop('name,buy_optionid,sell_optionid'.split(','),axis=1).sort_values('date,exdate,sell_strike,buy_strike'.split(',')).reset_index(drop=True) )

pd.options.display.float_format = '{:,.3f}'.format
np.set_printoptions(precision=3)
np.set_printoptions(linewidth=180)


if False:
  plimit = 3
  hprint("\nX_train"); display(om.X_train[:plimit].to_numpy())
  hprint("\ny_train"); display(om.y_train[:plimit])
  hprint("\nX_test"); display(om.X_test[:plimit])
  hprint("\ny_test"); display(om.y_test[:plimit])
  om.y_train.describe()



In [ ]:
#om.test_set

In [ ]:
om.X_test

In [ ]:
num_features = len(om.X_train.columns)
model = keras.models.Sequential([
  keras.layers.Flatten(input_shape=[1, num_features]),
  keras.layers.Dense(num_features*2, activation="relu"),
  keras.layers.Dense(num_features, activation="relu"),
  keras.layers.Dense(1, activation="relu") # use a softmax of 2 to get some level of uncertainty
#  keras.layers.Dense(10, activation="softmax") 
])

fprint("notice this is overfit. the loss is 0 right away. ")

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
history = model.fit(om.X_train.to_numpy(), om.y_train.to_numpy(), epochs=30, validation_data=(om.X_test.to_numpy(), om.y_test.to_numpy()))

In [ ]:
X_new = om.X_test.copy()
y_proba = (100*model.predict(X_new)).round(0)
X_new['y_hat'] = y_proba
hprint("heres what the predicted outcome looks like for ALL OUTCOMES")
display(X_new.y_hat.describe())

fig, axes = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True)
X_new.y_hat.hist()
plt.suptitle("outcome ROI percent frequency -- ALL")
fig.text(0.5, 0.04, 'predicted outcome ROI', ha='center')
fig.text(0.04, 0.5, 'frequency', va='center', rotation='vertical')
plt.show()



hprint("\nheres what the predicted outcome looks like for ALL positive OUTCOMES")
fig, axes = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True)
X_new.query("y_hat>0").y_hat.hist()
plt.suptitle("outcome ROI percent frequency -- where outcome >0")
fig.text(0.5, 0.04, 'predicted outcome ROI', ha='center')
fig.text(0.04, 0.5, 'frequency', va='center', rotation='vertical')

In [ ]:
om.positions.columns


In [ ]:
X_new

In [ ]:
t.query("y_hat>0")

In [ ]:

all = om.positions.merge(X_new.y_hat, how='left', left_index=True, right_index=True)['date,exdate,stock_price,stock_close,outcome,y_hat,days,cost,max_profit,max_ROI_pc,change_pc'.split(',')]
invest_wrong = t.query("y_hat>0 and outcome<0")
invest_correct = t.query("y_hat>0 and outcome>0")
hprint(f"{len(invest_wrong)} wrong investment predictions")
display(invest_wrong)

hprint(f"{len(invest_correct)} correct investment predictions")
display(invest_correct)
